In [1]:
import pandas as pd
import numpy as np
file = r'data_set/3-6 jobs_51_info.csv'
jobs = pd.read_csv(file, header=0)
jobs.head()

,job,company,city,salary_min,salary_max
0,金融数据分析师/助理,华壹信融投资管理（北京）有限责任...,北京,8333.33,12500.0
1,a（0经验可培养）金融数据分析师/助理,深圳市中创荣投资有限公司,深圳,8000.00,10000.0
2,初级运营数据分析专员 助理,知才（上海）信息技术有限公司,上海,6000.00,8000.0
3,销售数据分析,上海品星互联网信息技术有限公司,北京,5000.00,9000.0
4,数据分析经理,美菜网,上海,13000.00,17000.0


In [40]:
# 字段信息总览
jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66766 entries, 0 to 66765
Data columns (total 5 columns):
job           66766 non-null object
company       66766 non-null object
city          64290 non-null object
salary_min    65457 non-null float64
salary_max    65462 non-null float64
dtypes: float64(2), object(3)
memory usage: 2.5+ MB


In [41]:
# 查看数据包含哪些城市
jobs['city'].unique()

array(['北京', '深圳', '上海', '武汉', '广州', '南京', '西安', '杭州', '成都', '苏州', nan],
      dtype=object)

In [42]:
# 创建布尔类型Series的方法有大小比较、isin()、isnull()、notnull()，以及它们之间的位运算。
jobs[jobs['city'] == '广州']['salary_min'].mean(skipna=True)

7196.770184910765

In [43]:
# value_counts()是Series类型自带的方法，如果在DataFrame中使用，需要具体指定某一列 。
jobs['city'].value_counts()

广州    15634
深圳    14448
上海    13027
北京     5093
杭州     4615
武汉     3682
成都     2811
南京     2165
苏州     1486
西安     1329
Name: city, dtype: int64

In [44]:
jobs.shape
# 只要这一行存在空值，则删除改行；axis=0用来表征是删除行（运算方向为列方向）
# 同步计算删除结果的维度，为63017行5列。
# 从结果看，有三千多列存在空值
jobs.dropna(axis=0).shape
# 整行均为空值，删除；否则保留
# 从结果看，不存在全部为空的值
jobs.dropna(how='all', axis=0).shape
# 删除指定列("city","salary_min")中包含缺失值的行
jobs.dropna(subset=['city', 'salary_min']).shape

(63017, 5)

In [45]:
# 其实从数据分析师的角度，空值也是一种数据。通常我们会对空值做一些标记，例如这里我们把city字段中存在空值的部分，填充为“其他”。
# 填充city字段空值为其他，设置inplace为True。该语句本身返回值为None，但是jobs本身发生了变化
jobs['city'].fillna('其他', inplace=True)
jobs['city'].value_counts()

广州    15634
深圳    14448
上海    13027
北京     5093
杭州     4615
武汉     3682
成都     2811
其他     2476
南京     2165
苏州     1486
西安     1329
Name: city, dtype: int64

In [46]:
# 对于salary_min字段，我们这里以均值填充，即以该字段的平均值来进行填充。
jobs['salary_min'].fillna(jobs['salary_min'].mean())
jobs.groupby(['city']).apply(lambda g: g.fillna(g.mean()))
# 对于 salary_max字段，我们这里以10000-20000之间的随机值填充。
jobs['salary_max'].fillna(np.random.randint(10000, 20000))
# 或者我们可以以用前一个非缺失值进行填充
jobs['salary_max'].fillna(method='ffill')

0        12500.0
1        10000.0
2         8000.0
3         9000.0
4        17000.0
          ...   
66761     5000.0
66762    10000.0
66763    30000.0
66764    50000.0
66765    16000.0
Name: salary_max, Length: 66766, dtype: float64

In [47]:
# insert函数至少需要传入3个参数：
# 第一个参数表示插入列的具体位置
# 第二参数表示插入列的列名
# 第三个参数表示插入列的赋值.
jobs.insert(1, 'data_engineer',
            jobs['job'].apply(lambda x: 'y' if '数据分析' in x else 'n'))
jobs

,job,data_engineer,company,city,salary_min,salary_max
0,金融数据分析师/助理,y,华壹信融投资管理（北京）有限责任...,北京,8333.33,12500.0
1,a（0经验可培养）金融数据分析师/助理,y,深圳市中创荣投资有限公司,深圳,8000.00,10000.0
2,初级运营数据分析专员 助理,y,知才（上海）信息技术有限公司,上海,6000.00,8000.0
3,销售数据分析,y,上海品星互联网信息技术有限公司,北京,5000.00,9000.0
4,数据分析经理,y,美菜网,上海,13000.00,17000.0
...,...,...,...,...,...,...
66761,运营支持岗,n,九州通医药集团物流有限公司,武汉,3000.00,5000.0
66762,新媒体运营专员,n,深圳市摩天之星企业管理有限公司龙...,深圳,8000.00,10000.0
66763,电商运营总监,n,福建特家商业管理有限公司,广州,20000.00,30000.0
66764,总经理/CEO,n,福建特家商业管理有限公司,广州,30000.00,50000.0


In [49]:
# 这里通过重新设置列索引的方式间接实现了排序
jobs.reindex(
    columns='job data_engineer company salary_min salary_max cite'.split())

,job,data_engineer,company,salary_min,salary_max,cite
0,金融数据分析师/助理,y,华壹信融投资管理（北京）有限责任...,8333.33,12500.0,NaN
1,a（0经验可培养）金融数据分析师/助理,y,深圳市中创荣投资有限公司,8000.00,10000.0,NaN
2,初级运营数据分析专员 助理,y,知才（上海）信息技术有限公司,6000.00,8000.0,NaN
3,销售数据分析,y,上海品星互联网信息技术有限公司,5000.00,9000.0,NaN
4,数据分析经理,y,美菜网,13000.00,17000.0,NaN
...,...,...,...,...,...,...
66761,运营支持岗,n,九州通医药集团物流有限公司,3000.00,5000.0,NaN
66762,新媒体运营专员,n,深圳市摩天之星企业管理有限公司龙...,8000.00,10000.0,NaN
66763,电商运营总监,n,福建特家商业管理有限公司,20000.00,30000.0,NaN
66764,总经理/CEO,n,福建特家商业管理有限公司,30000.00,50000.0,NaN


In [50]:
# 单列删除则较为简单，利用del函数即可。例如这里我们删除上面刚刚生成的data_engineer字段
del jobs['data_engineer']

In [51]:
# 这里用axis=1表示删除列
# inplace默认为False
jobs.drop(['salary_min', 'salary_max'], inplace=False, axis=1)

,job,company,city
0,金融数据分析师/助理,华壹信融投资管理（北京）有限责任...,北京
1,a（0经验可培养）金融数据分析师/助理,深圳市中创荣投资有限公司,深圳
2,初级运营数据分析专员 助理,知才（上海）信息技术有限公司,上海
3,销售数据分析,上海品星互联网信息技术有限公司,北京
4,数据分析经理,美菜网,上海
...,...,...,...
66761,运营支持岗,九州通医药集团物流有限公司,武汉
66762,新媒体运营专员,深圳市摩天之星企业管理有限公司龙...,深圳
66763,电商运营总监,福建特家商业管理有限公司,广州
66764,总经理/CEO,福建特家商业管理有限公司,广州


In [52]:
# 在Pandas中，排序分为对值的排序和对索引的排序，其中每个排序又可以根据具体的场景，可以分为行方向排序和列方向排序。
# DataFrame.sort_index(axis=0, ascending=True, inplace=False, kind='quicksort')
jobs.sort_index(axis=0, ascending=False)

,job,company,city,salary_min,salary_max
66765,会员商城运营管理师,统一企业（中国）投资有限公司,上海,8000.00,16000.0
66764,总经理/CEO,福建特家商业管理有限公司,广州,30000.00,50000.0
66763,电商运营总监,福建特家商业管理有限公司,广州,20000.00,30000.0
66762,新媒体运营专员,深圳市摩天之星企业管理有限公司龙...,深圳,8000.00,10000.0
66761,运营支持岗,九州通医药集团物流有限公司,武汉,3000.00,5000.0
...,...,...,...,...,...
4,数据分析经理,美菜网,上海,13000.00,17000.0
3,销售数据分析,上海品星互联网信息技术有限公司,北京,5000.00,9000.0
2,初级运营数据分析专员 助理,知才（上海）信息技术有限公司,上海,6000.00,8000.0
1,a（0经验可培养）金融数据分析师/助理,深圳市中创荣投资有限公司,深圳,8000.00,10000.0


In [53]:
# DataFrame.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
jobs.sort_values(by=['city', 'company'], axis=0, ascending=[True, False])

,job,company,city,salary_min,salary_max
859,数据分析师,（CCE GROUP）上海程迈文化传播有...,上海,15000.0,20000.0
50689,资深媒介专员,（CCE GROUP）上海程迈文化传播有...,上海,6000.0,8000.0
23092,产品运营经理（教育行业）,龙的股份,上海,5000.0,8000.0
4113,教务助理/教务人员,鼓动商贸（上海）有限公司,上海,4000.0,7000.0
27576,教务专员,鼓动商贸（上海）有限公司,上海,5000.0,6500.0
...,...,...,...,...,...
39420,商品经理,ABC童装童鞋,西安,6000.0,8000.0
39717,急聘商品专员,ABC童装童鞋,西安,4000.0,6000.0
39719,商品主管,ABC童装童鞋,西安,4000.0,8000.0
41307,直营商品专员,ABC童装童鞋,西安,4000.0,6000.0


In [54]:
# DataFrame.drop_duplicates(subset=['A','B'],keep='first',inplace=True)
# subset用来指定特定的列，字符串或者列表类型；默认所有列，即默认全字段去重；
# inplace字段同上。这里我们以job和company字段进行联合去重，即我们认为同一公司的同一职位只招聘一次：
jobs.drop_duplicates(subset=['job', 'company'], keep='first')

,job,company,city,salary_min,salary_max
0,金融数据分析师/助理,华壹信融投资管理（北京）有限责任...,北京,8333.33,12500.0
1,a（0经验可培养）金融数据分析师/助理,深圳市中创荣投资有限公司,深圳,8000.00,10000.0
2,初级运营数据分析专员 助理,知才（上海）信息技术有限公司,上海,6000.00,8000.0
3,销售数据分析,上海品星互联网信息技术有限公司,北京,5000.00,9000.0
4,数据分析经理,美菜网,上海,13000.00,17000.0
...,...,...,...,...,...
66761,运营支持岗,九州通医药集团物流有限公司,武汉,3000.00,5000.0
66762,新媒体运营专员,深圳市摩天之星企业管理有限公司龙...,深圳,8000.00,10000.0
66763,电商运营总监,福建特家商业管理有限公司,广州,20000.00,30000.0
66764,总经理/CEO,福建特家商业管理有限公司,广州,30000.00,50000.0


In [59]:
# 查看中位数
np.percentile(jobs['salary_min'].dropna().values, 50)
# 查看TOP 10%
np.percentile(jobs['salary_min'].dropna().values, 90)
# 查看TOP 1%
np.percentile(jobs['salary_min'].dropna().values, 99)
# 查看最大值
jobs['salary_min'].max()
# 查看最小值
jobs['salary_min'].min()

1100.0

In [63]:
# 计算标准差
jobs['salary_min'].std()
# 计算均值
jobs['salary_min'].mean()
# 计算最大值的偏离程度
(jobs['salary_min'].max() -
 jobs['salary_min'].mean()) / jobs['salary_min'].std()
# 计算最小值的偏离程度
(jobs['salary_min'].mean() -
 jobs['salary_min'].min()) / jobs['salary_min'].std()

1.2459869483634876